Make sure we have a GPU :

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

# Fetch repo and general setup

In [ ]:
!pip install ultralytics
!git clone https://github.com/pekatour/Charlie-DB.git
!cp ./Charlie-DB/data.yaml data.yaml
!mkdir datasets

## ⚠️ Choose one Dataset

Dataset 1 : full images

In [ ]:
!cp -r ./Charlie-DB/dataset datasets/dataset

Dataset 2 : imagettes

In [ ]:
!cp -r ./Charlie-DB/dataset-imagettes datasets/dataset

Run this only if you want to reimport a dataset

In [ ]:
!rm -rf datasets/
!mkdir datasets

# Train

In [ ]:
# Import necessary libraries
from ultralytics import YOLO

# Load a pretrained YOLO model
model = YOLO('yolo11n.pt') # Or any other suitable model

# Train the model
results = model.train(data='./data.yaml', epochs=50, imgsz=300) # Adjust parameters as needed

In [ ]:
# Display results again
metrics = model.val()
print(metrics.box.map)

# Check

## Predict on the test folder

In [ ]:
model = YOLO("runs/detect/train4/weights/best.pt")

# Run inference on test images and save results
results = model.predict(source="./datasets/dataset/test/images", save=True)

### Display some of the predictions of the test folder

In [ ]:
import glob
import random
import os
from IPython.display import Image, display

directory = './runs/detect/predict'
image_files = [f for f in os.listdir(directory) if f.endswith('.jpg')]

# Get full paths
image_paths = [os.path.join(directory, f) for f in image_files]

sample_size = min(10, len(image_paths))
for image_path in random.sample(image_paths, sample_size):
  display(Image(filename=image_path, height=400))
  print('\n')

## Predict on a full image with a threshold

In [ ]:
!cp Charlie-DB/full_image_predict.py .
!cp Charlie-DB/visualize_annotations.py .
!cp Charlie-DB/vis_bbox.py .

In [ ]:
from full_image_predict import full_image_predict

full_image_predict(model, "./Charlie-DB/original-images/6-3-0.jpg", 300)

Display the full image with predictions

In [ ]:
# prompt: display the image located in folder named full_image_output

from IPython.display import Image, display
import os

directory = './full_image_output'
image_files = [f for f in os.listdir(directory) if f.lower().endswith(('.jpg'))]

# Get full paths
image_paths = [os.path.join(directory, f) for f in image_files]

for image_path in image_paths:
  display(Image(filename=image_path, height=400))
  print('\n')

# Export trained model

In [ ]:
from google.colab import files
!zip -r train.zip runs/detect/train4/
files.download('train.zip')

In [ ]:
# Optionnel
model.export(format="onnx")